In [46]:
# import litellm
# client = litellm.LiteLLM(
#     base_url="https://mistral-large-vldor-serverless.swedencentral.inference.ai.azure.com/",
#     api_key="#### )
# response = client.chat.completions.create(
#     messages=[{"content": "What do you think about cHAT gpt? Do you think you are better?", "role": "user"}],
#     model="Mistral-large-mbbjt",
#     custom_llm_provider="custom_openai",

# )
# print(response.choices[0].message.content)


In [48]:
import pandas as pd
data = pd.read_csv("LABED2_dat.csv")

In [49]:
insert_position = data.columns.get_loc('label_4') + 1

# Insert new columns to the right of 'label_4'
data.insert(insert_position, 'mistral_label', None)
data.insert(insert_position + 1, 'mistral_label_n', None)

In [50]:
data

,Unnamed: 0.1,Unnamed: 0,eventid,eventid.1,summary,weaptype1_txt,city,attacktype1_txt,targtype1_txt,label_3,label_4,mistral_label,mistral_label_n,scraped,error
0,0,0,200402020010,200402020010,The office of Senate majority leader Bill Fris...,Biological,Washington,Facility/Infrastructure Attack,Government (General),6,NaN,None,None,False,False
1,1,1,200110290003,200110290003,"In one of several anthrax-related incidents, K...",Biological,New York City,Unarmed Assault,Business,7,6.0,None,None,True,False
2,2,2,200110150004,200110150004,Six employees were tested for anthrax exposure...,Biological,Reno,Unarmed Assault,Business,6,NaN,None,None,False,False
3,3,3,200111190003,200111190003,A letter containing anthrax spores was deliver...,Biological,Santiago,Unarmed Assault,Business,6,NaN,None,None,False,False
4,4,4,201011160004,201011160004,"On Tuesday, at the University of Los Angeles, ...",Biological,Los Angeles,Unarmed Assault,Educational Institution,6,NaN,None,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,1281,1281,201811230001,201811230001,An assailant in a vehicle attempted to run ove...,Vehicle (not to include vehicle-borne explosiv...,Los Angeles,Unarmed Assault,Religious Figures/Institutions,3,NaN,None,None,False,False
1282,1282,1282,201710310017,201710310017,An assailant driving a Home Depot rental truck...,Vehicle (not to include vehicle-borne explosiv...,New York City,Unarmed Assault,Tourists,5,NaN,None,None,False,False
1283,1283,1283,201408040018,201408040018,An assailant stole a construction excavator an...,Vehicle (not to include vehicle-borne explosiv...,Jerusalem,Unarmed Assault,Transportation,6,NaN,None,None,False,False
1284,1284,1284,201504150078,201504150078,An assailant rammed his vehicle into a bus sto...,Vehicle (not to include vehicle-borne explosiv...,Jerusalem,Unarmed Assault,Transportation,3,NaN,None,None,False,False


In [51]:
import pandas as pd
import litellm

# Initialize the LiteLLM client
client = litellm.LiteLLM(
    base_url="https://mistral-large-vldor-serverless.swedencentral.inference.ai.azure.com/",
    api_key="###)

# Define the system prompt
system_prompt = """
Rate the sophistication of a terrorist incident from the Global Terrorism Database (GTD) using the detailed criteria below. Each level describes the extent of planning, resource management, risk handling, coordination, and technological use in the attack:

1 - Very Low: Primitive tactics, minimal planning. Example: Lone-wolf attack with basic, handmade devices.
2 - Low: Basic coordination, simple tactics, limited tech use. Example: Small group with readily available weapons.
3 - Slightly Low: Limited planning, basic risk management, some tech use. Example: Coordinated attack with basic encryption.
4 - Moderate: Advanced planning, standard resource use, moderate tech. Example: Attack with encrypted communications.
5 - Slightly High: Thorough planning, good resource management, some advanced tech. Example: Multi-stage attack with cyber elements.
6 - High: Well-coordinated, advanced tech, and thorough planning. Example: High-grade weaponry with tech-enabled coordination.
7 - Very High: State-of-the-art tech, exceptional planning, flawless execution. Example: Highly coordinated international operation with groundbreaking tactics.

The following data fields are critical for your evaluation:
- City: Location of the attack, providing context to the operational environment.
- Attack Type: Describes the method of attack, which indicates the level of planning and execution.
- Target Type: Shows the focus of the attack, highlighting strategic choices and objectives.
- Weapon Details: Gives insights into the technological and tactical sophistication of the attack.
- Summary: Offers a brief overview of the incident, which can provide additional context on complexity.

Provide only the numerical value corresponding to your assessment based on these definitions, without any additional explanation.
"""

user_prompt_template = "Please assess the sophistication of the following attack based on the provided criteria: "

# Function to generate the sophistication rating
def generate_sophistication_rating(row):
    if pd.isna(row['label_4']):
        return row['mistral_label'], row['mistral_label_n']  # Skip if label_4 is NaN
    
    incident_details = f"""
    City: {row['city']}
    Attack Type: {row['attacktype1_txt']}
    Target Type: {row['targtype1_txt']}
    Weapon Details: {row['weaptype1_txt']}
    Summary: {row['summary']}
    """
    full_prompt = f"{system_prompt}\n\n{user_prompt_template}{incident_details}"
    
    response = client.chat.completions.create(
        messages=[{"content": full_prompt, "role": "user"}],
        model="Mistral-large-mbbjt",
        custom_llm_provider="custom_openai",
    )
    
    rating = response.choices[0].message.content.strip()
    
    # Extract only the numerical rating from the response
    numerical_rating = ''.join(filter(str.isdigit, rating))
    return rating, numerical_rating


# Load the data into a DataFrame
df = pd.DataFrame(data)

# Apply the function to each row where label_4 is not NaN
df[['mistral_label', 'mistral_label_n']] = df.apply(
    lambda row: pd.Series(generate_sophistication_rating(row)) if pd.notna(row['label_4']) else pd.Series([row['mistral_label'], row['mistral_label_n']]),
    axis=1
)

display(df)


,Unnamed: 0.1,Unnamed: 0,eventid,eventid.1,summary,weaptype1_txt,city,attacktype1_txt,targtype1_txt,label_3,label_4,mistral_label,mistral_label_n,scraped,error
0,0,0,200402020010,200402020010,The office of Senate majority leader Bill Fris...,Biological,Washington,Facility/Infrastructure Attack,Government (General),6,NaN,None,None,False,False
1,1,1,200110290003,200110290003,"In one of several anthrax-related incidents, K...",Biological,New York City,Unarmed Assault,Business,7,6.0,"6 - High: Well-coordinated, advanced tech, and...",6,True,False
2,2,2,200110150004,200110150004,Six employees were tested for anthrax exposure...,Biological,Reno,Unarmed Assault,Business,6,NaN,None,None,False,False
3,3,3,200111190003,200111190003,A letter containing anthrax spores was deliver...,Biological,Santiago,Unarmed Assault,Business,6,NaN,None,None,False,False
4,4,4,201011160004,201011160004,"On Tuesday, at the University of Los Angeles, ...",Biological,Los Angeles,Unarmed Assault,Educational Institution,6,NaN,None,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,1281,1281,201811230001,201811230001,An assailant in a vehicle attempted to run ove...,Vehicle (not to include vehicle-borne explosiv...,Los Angeles,Unarmed Assault,Religious Figures/Institutions,3,NaN,None,None,False,False
1282,1282,1282,201710310017,201710310017,An assailant driving a Home Depot rental truck...,Vehicle (not to include vehicle-borne explosiv...,New York City,Unarmed Assault,Tourists,5,NaN,None,None,False,False
1283,1283,1283,201408040018,201408040018,An assailant stole a construction excavator an...,Vehicle (not to include vehicle-borne explosiv...,Jerusalem,Unarmed Assault,Transportation,6,NaN,None,None,False,False
1284,1284,1284,201504150078,201504150078,An assailant rammed his vehicle into a bus sto...,Vehicle (not to include vehicle-borne explosiv...,Jerusalem,Unarmed Assault,Transportation,3,NaN,None,None,False,False


In [70]:
df['mistral_label_n'].value_counts()

mistral_label_n
2.0    112
3.0     40
4.0     27
5.0     17
6.0     10
1.0      3
7.0      2
5        1
Name: count, dtype: int64

In [75]:
def extract_numerical_rating(response_text):
    numerical_rating = ''
    if response_text is not None:
        for word in response_text.split():
            if word.isdigit():
                numerical_rating = word
                break
    return numerical_rating

df['mistral_label_n'] = df['mistral_label'].apply(extract_numerical_rating)

In [79]:
# Save DataFrame to pickle file
df.to_pickle("final_labels.pkl")

In [81]:
df = pd.read_pickle("final_labels.pkl")